# 🧠 CORTEX v4.0 — NotebookLM Auto-Synthesizer

Este notebook es el motor de inteligencia que prepara la memoria de **CORTEX** para ser ingerida por **NotebookLM**.

### Funcionalidades Avanzadas:
1. **Detección de Clusters de Incertidumbre**: Encuentra lagunas de conocimiento o hechos no verificados.
2. **Análisis de Huérfanos**: Identifica entidades mencionadas pero no relacionadas.
3. **Síntesis para NotebookLM**: Genera archivos Markdown estructurados por "Dominio de Conocimiento".

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# Configuración de rutas
CORTEX_DB_PATH = Path.home() / ".cortex" / "cortex.db"
OUTPUT_DIR = Path("../notebooklm_sources")
OUTPUT_DIR.mkdir(exist_ok=True)


def get_connection():
    return sqlite3.connect(str(CORTEX_DB_PATH))


print(f"📌 Conectando a CORTEX DB en: {CORTEX_DB_PATH}")

## 1. Detección de Incertidumbre y Huérfanos
Analizamos la base de datos buscando:
- Facts con confianza baja (`stated` vs `verified`).
- Entidades sin relaciones (orphans).

In [ ]:
with get_connection() as conn:
    # 1. Hechos no verificados
    unverified_query = """
    SELECT project, COUNT(*) as count 
    FROM facts 
    WHERE confidence != 'verified' AND valid_until IS NULL
    GROUP BY project
    """
    df_unverified = pd.read_sql_query(unverified_query, conn)

    # 2. Entidades huérfanas (sin relaciones)
    orphan_query = """
    SELECT name, entity_type, project, mention_count
    FROM entities
    WHERE id NOT IN (SELECT source_entity_id FROM entity_relations) 
      AND id NOT IN (SELECT target_entity_id FROM entity_relations)
    ORDER BY mention_count DESC
    """
    df_orphans = pd.read_sql_query(orphan_query, conn)

print("⚠️ Proyectos con hechos no verificados:")
display(df_unverified)

print("\n🕵️ Entidades Huérfanas (posibles lagunas de contexto):")
display(df_orphans.head(10))

## 2. Generación Inteligente de Fuentes
Generamos un archivo Markdown que añade un bloque de **"RECOMENDACIONES DE INVESTIGACIÓN"** para que NotebookLM sepa dónde focalizarse.

In [ ]:
def generate_enhanced_sources():
    with get_connection() as conn:
        # Cargar todos los facts activos
        query = "SELECT project, fact_type, content, confidence, tags FROM facts WHERE valid_until IS NULL"
        df = pd.read_sql_query(query, conn)

    projects = df["project"].unique()

    for project in projects:
        proj_df = df[df["project"] == project]
        filename = OUTPUT_DIR / f"{project}_knowledge.md"

        # Detectar lagunas específicas para este proyecto
        proj_orphans = df_orphans[df_orphans["project"] == project]["name"].tolist()
        unverified_count = df_unverified[df_unverified["project"] == project]["count"].sum()

        with open(filename, "w", encoding="utf-8") as f:
            f.write(f"# 🧠 CORTEX Domain: {project.upper()}\n\n")

            if unverified_count > 0 or proj_orphans:
                f.write(f"## 🔍 NOTAS DE INVESTIGACIÓN (CRÍTICO)\n")
                f.write(
                    f"> NotebookLM: He detectado las siguientes lagunas en CORTEX para este proyecto.\n"
                )
                if unverified_count > 0:
                    f.write(
                        f"- Hay **{unverified_count}** hechos sin verificar que requieren validación lógica.\n"
                    )
                if proj_orphans:
                    f.write(
                        f"- Las siguientes entidades carecen de conexiones relacionales: {', '.join(proj_orphans[:5])}.\n"
                    )
                f.write("\n")

            f.write(f"## Base de Conocimiento\n")
            for ftype in proj_df["fact_type"].unique():
                f.write(f"### {ftype.capitalize()}\n")
                type_df = proj_df[proj_df["fact_type"] == ftype]
                for _, row in type_df.iterrows():
                    f.write(f"- **{row['content']}** (Confid: {row['confidence']})\n")

        print(f"✅ {filename} sintetizado con éxito.")


generate_enhanced_sources()

## 3. Resumen Ejecutivo de Memoria
Generamos un dashboard visual rápido de la salud de CORTEX.

In [ ]:
with get_connection() as conn:
    stats_query = "SELECT fact_type, COUNT(*) as count FROM facts WHERE valid_until IS NULL GROUP BY fact_type"
    df_stats = pd.read_sql_query(stats_query, conn)

plt.figure(figsize=(10, 5))
plt.bar(df_stats["fact_type"], df_stats["count"], color="skyblue")
plt.title("Distribución de Facts Activos en CORTEX")
plt.xlabel("Tipo de Hecho")
plt.ylabel("Cantidad")
plt.show()